<a id='0'></a>
# 0. Minimum required packages and settings

<a id='0.1'></a>
## 0.1 import required packages

In [1]:
import os,sys
import numpy as np
import pickle
from tqdm.notebook import tqdm

sys.path.append(os.path.abspath(os.path.join(r"..", r".")))

import source as ia

print(os.getpid()) # print this so u can terminate through cmd / task-manager

382040


<a id='0.2'></a>
## 0.2 parameters for plotting

In [2]:
# Required plotting setting
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')
_font_size = 7.5

In [3]:
# Required plotting parameters
from source.figure_tools import _dpi,_single_col_width,_double_col_width,_single_row_height,_ref_bar_length, _ticklabel_size,_ticklabel_width,_font_size

## 0.gc Directories

In [4]:
# please change data_folder into the folder for downloaded dataset
data_folder = os.path.abspath(os.path.join(r"..", r"data", r"zenodo_dl"))
print(f"data_folder: {data_folder}")

# [GC+] For saving partial progress
progress_folder = os.path.abspath(os.path.join(r"..", r"data", r"in_progress"))
print(f"progress_folder: {progress_folder}")
os.makedirs(progress_folder, exist_ok=True)
os.makedirs(os.path.join(progress_folder, '3d_coords'), exist_ok=True)
os.makedirs(os.path.join(progress_folder, 'distances'), exist_ok=True)

# [GC+] For saving final counts matrices
counts_folder = os.path.abspath(os.path.join(r"..", r"data", r"counts"))
print(f"counts_folder: {counts_folder}")
os.makedirs(counts_folder, exist_ok=True)

# # figure folder
# # please specify location to save images
# results_folder = os.path.abspath(os.path.join(r"..", r"results"))
# parent_figure_folder = os.path.abspath(os.path.join(results_folder, "figures"))
# os.makedirs(results_folder, exist_ok=True)
# os.makedirs(parent_figure_folder, exist_ok=True)
# print(f"results_folder: {results_folder}")
# print(f"parent_figure_folder: {parent_figure_folder}")

data_folder: /net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/zenodo_dl
progress_folder: /net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress
counts_folder: /net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/counts


In [5]:
import re

# please update as needed
files_3dcoords = {
    'chr21': {
        'rep1': os.path.join(data_folder, 'chrom_trace',  'chr21.rep1.tsv'),
        'rep2': os.path.join(data_folder, 'chrom_trace', 'chr21.rep2_cell_cycle.tsv')},
    'chr2': {
        'rep1': os.path.join(data_folder, 'chrom_trace',  'chr2.rep1.tsv'),
        'rep2': os.path.join(data_folder, 'chrom_trace', 'chr2.rep2_p_arm.tsv')}}


files_progress = {chrom: {} for chrom in files_3dcoords.keys()}

for chrom in files_3dcoords.keys():
    files_progress[chrom]['dir_coords3d'] = os.path.join(progress_folder, '3d_coords', chrom)
    os.makedirs(files_progress[chrom]['dir_coords3d'], exist_ok=True)
    
    files_progress[chrom]['rep1'] = os.path.basename(files_3dcoords[chrom]['rep1']).replace('.tsv', '')
    files_progress[chrom]['rep2'] = os.path.basename(files_3dcoords[chrom]['rep2']).replace('.tsv', '')
    
    for desc in ('distances.per_cell', 'distances.median', 'distances.mean', 'contacts.mean.cutoff500', 'contacts.per_cell.cutoff500'):
        files_progress[chrom][desc] = os.path.join(progress_folder, 'distances', f'{chrom}.{desc}.npy')
        

files_progress['chr21']['proceed_with'] = 'chr21'
files_progress['chr2']['proceed_with'] = files_progress['chr2']['rep1']

Please visit zenodo with DOI:10.5281/zenodo.3928890

## 0.gc Function for working with sc distances

In [47]:
def process_sc_distances(filenames_chrom, distvec_per_cell=None, dis_mean=True, dis_median=True, contact_th=500):
    from scipy.spatial.distance import pdist, squareform
    
    # distvec_per_cell_file = filenames_chrom['distances.per_cell']
    # median_distance_map_file = filenames_chrom['distances.median']
    # mean_distance_map_file = filenames_chrom['distances.mean']
    # contacts_mean_file = filenames_chrom[f'contacts.mean.cutoff{contact_th}']
    # contacts_percell_file = filenames_chrom[f'contacts.per_cell.cutoff{contact_th}']

    if distvec_per_cell is None:
        print('Loading sc distances...')
        distvec_per_cell = np.load(filenames_chrom['distances.per_cell'])

    # print("Converting distance vectors to 'squareform' matrices...")
    # distmap_per_cell = np.stack([squareform(distvec) for distvec in distvec_per_cell])

    if dis_median:
        print('Generate median distances across cells...')
        median_distance_map = squareform(np.nanmedian(distvec_per_cell, axis=0))
        np.save(filenames_chrom['distances.median'], median_distance_map) # [GC+] SAVE in-progress files

    if dis_mean: # [GC+] generate MEAN distance map
        print('Generate mean distances across cells...')
        mean_distance_map = squareform(np.nanmean(distvec_per_cell, axis=0))
        np.save(filenames_chrom['distances.mean'], mean_distance_map) # [GC+] SAVE in-progress files

    if contact_th is not None:
        print('Generate contacts...')
        contacts_per_cell = (distvec_per_cell<contact_th).astype(int)
        print('                 ...saving contacts<thresh per cell') # [GC+] generate contacts<thresh PER-CELL
        np.save(filenames_chrom[f'contacts.per_cell.cutoff{contact_th}'], contacts_per_cell)
        # contacts_mean = np.sum(contacts_per_cell, axis=0) / np.sum(np.isnan(distvec_per_cell)==False, axis=0)
        contacts_mean = squareform(np.nanmean(contacts_per_cell, axis=0))
        print('                 ...saving mean contacts across cells')
        np.save(filenames_chrom[f'contacts.mean.cutoff{contact_th}'], contacts_mean) # [GC+] SAVE in-progress files

    print('Done!')
    
    # print("Converting contact matrices to vectors...") # Can be done with squareform or with triu_indices
    # # contacts_per_cell = np.stack([squareform(contact_matrix) for contact_matrix in contacts_per_cell]) # equivalent to the below
    # nbins = contacts_per_cell[-1]
    # triu_idx = np.triu_indices(nbins, 1)
    # contacts_per_cell = contacts_per_cell[:, triu_idx[0], triu_idx[1]]
    

## Temp

In [ ]:
# confirming that squareform is the same as using np.triu etc

from scipy.spatial.distance import pdist, squareform

size = 40
arr = np.arange(size ** 2).reshape(size, size)
arr = arr + arr.T
np.fill_diagonal(arr, 0)

arr_vec1 = squareform(arr)
triu_idx = np.triu_indices(size, 1)
arr_vec2 = arr[triu_idx]
print(np.array_equal(arr_vec1, arr_vec2))

arr_vec_arr1 = squareform(arr_vec1)
arr_vec_arr2 = np.zeros([size, size])
arr_vec_arr2[np.triu_indices(size, 1)] = arr_vec2
arr_vec_arr2 += arr_vec_arr2.T
print(np.array_equal(arr_vec_arr1, arr_vec_arr2))

arrB = arr + 1
arrC = arr + 1
np.fill_diagonal(arrB, 0)
np.fill_diagonal(arrC, 0)
arrs = np.stack([arr, arrB, arrC])

arrs_vec1 = np.stack([squareform(x) for x in arrs])
arrs_vec2 = arrs[:, triu_idx[0], triu_idx[1]]
print(np.array_equal(arrs_vec1, arrs_vec2))

arrs.shape, arrs_vec2.shape

<a id='1'></a>
# Chromosome 21

<a id='1.gc'></a>
## 1.gc Filenames

In [7]:
chrom = 'chr21'
rep1_filename = files_3dcoords[chrom]['rep1']
rep2_filename = files_3dcoords[chrom]['rep2']
rep1 = files_progress[chrom]['rep1']
rep2 = files_progress[chrom]['rep2']
print(chrom); print(rep1); print(rep2)

dir_3d_coords =  files_progress[chrom]['dir_coords3d']
distmap_list_file = files_progress[chrom]['distances.per_cell']
median_distance_map_file = files_progress[chrom]['distances.median']
mean_distance_map_file = files_progress[chrom]['distances.mean']
contacts_mean_c500_file = files_progress[chrom]['contacts.mean.cutoff500']
contacts_percell_c500_file = files_progress[chrom]['contacts.per_cell.cutoff500']

chr21
chr21.rep1
chr21.rep2_cell_cycle


<a id='1.1'></a>
## 1.1 load chr21 replicate 1 

(without cell cycle information)

In [106]:
# load from file and extract info
import csv
rep1_info_dict = {}
with open(rep1_filename, 'r') as _handle:
    _reader = csv.reader(_handle, delimiter='\t', quotechar='|')
    _headers = next(_reader)
    print(_headers)
    # create keys for each header
    for _h in _headers:
        rep1_info_dict[_h] = []
    # loop through content
    for _contents in _reader:
        for _h, _info in zip(_headers,_contents):
            rep1_info_dict[_h].append(_info)

# clean up information
data_rep1 = {'params':{}}

# clean up genomic coordiantes
region_names = np.array([_n for _n in sorted(np.unique(rep1_info_dict['Genomic coordinate']), 
                                             key=lambda s:int(s.split(':')[1].split('-')[0]))])
region_starts = np.array([int(_n.split(':')[1].split('-')[0]) for _n in region_names])
region_ends = np.array([int(_n.split(':')[1].split('-')[1]) for _n in region_names])[np.argsort(region_starts)]
region_starts = np.sort(region_starts)

mid_positions = ((region_starts + region_ends)/2).astype(int)
mid_positions_Mb = np.round(mid_positions / 1e6, 5) 
start_position_Mb = np.round(region_starts / 1e6, 5) 
end_position_Mb = np.round(region_ends / 1e6, 5) 

# clean up chrom copy number
chr_nums = np.array([int(_info) for _info in rep1_info_dict['Chromosome copy number']])
chr_ids, region_cts = np.unique(chr_nums, return_counts=True)
dna_zxys_list = [[[] for _start in region_starts] for _id in chr_ids]

# clean up zxy
for _z,_x,_y,_reg_info, _cid in tqdm(zip(rep1_info_dict['Z(nm)'],rep1_info_dict['X(nm)'],\
                                         rep1_info_dict['Y(nm)'],rep1_info_dict['Genomic coordinate'],\
                                         rep1_info_dict['Chromosome copy number'])):
    # get chromosome inds
    _cid = int(_cid)
    _cind = np.where(chr_ids == _cid)[0][0]
    
    # get region indices
    _start = int(_reg_info.split(':')[1].split('-')[0])
    _rind = np.where(region_starts==_start)[0][0]
    
    dna_zxys_list[_cind][_rind] = np.array([float(_z),float(_x), float(_y)])

# merge together
dna_zxys_list = np.array(dna_zxys_list)
data_rep1['chrom_ids'] = chr_ids
data_rep1['region_names'] = region_names
data_rep1['mid_position_Mb'] = mid_positions_Mb
data_rep1['start_position_Mb'] = start_position_Mb
data_rep1['end_position_Mb'] = end_position_Mb
data_rep1['dna_zxys'] = dna_zxys_list

# clean up tss and transcription
if 'Gene names' in rep1_info_dict:
    import re
    # first extract number of genes
    gene_names = []
    for _gene_info, _trans_info, _tss_coord in zip(rep1_info_dict['Gene names'],
                                                   rep1_info_dict['Transcription'],
                                                   rep1_info_dict['TSS ZXY(nm)']):
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            for _gene in _genes:
                if _gene not in gene_names:
                    gene_names.append(_gene)
    print(f"{len(gene_names)} genes exist in this dataset.")
    
    # initialize gene and transcription
    tss_zxys_list = [[[] for _gene in gene_names] for _id in chr_ids]
    transcription_profiles = [[[] for _gene in gene_names] for _id in chr_ids]
    # loop through to get info
    for _cid, _gene_info, _trans_info, _tss_locations in tqdm(zip(rep1_info_dict['Chromosome copy number'],
                                                                  rep1_info_dict['Gene names'],
                                                                  rep1_info_dict['Transcription'],
                                                                  rep1_info_dict['TSS ZXY(nm)'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        # process if there are genes in this region:
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            _transcribes = _trans_info.split(';')[:-1]
            _tss_zxys = _tss_locations.split(';')[:-1]
            for _gene, _transcribe, _tss_zxy in zip(_genes, _transcribes, _tss_zxys):
                # get gene index
                _gind = gene_names.index(_gene)
                # get transcription profile
                if _transcribe == 'on':
                    transcription_profiles[_cind][_gind] = True
                else:
                    transcription_profiles[_cind][_gind] = False
                # get coordinates
                _tss_zxy = np.array([np.float(_c) for _c in re.split(r'\s+', _tss_zxy.split('[')[1].split(']')[0]) if _c != ''])
                tss_zxys_list[_cind][_gind] = _tss_zxy
                
    tss_zxys_list = np.array(tss_zxys_list)
    transcription_profiles = np.array(transcription_profiles)
    data_rep1['gene_names'] = gene_names
    data_rep1['tss_zxys'] = tss_zxys_list
    data_rep1['trans_pfs'] = transcription_profiles

# clean up cell_cycle states
if 'Cell cycle state' in rep1_info_dict:
    cell_cycle_types = np.unique(rep1_info_dict['Cell cycle state'])
    cell_cycle_flag_dict = {_k:[[] for _id in chr_ids] for _k in cell_cycle_types if _k != 'ND'}
    for _cid, _state in tqdm(zip(rep1_info_dict['Chromosome copy number'],rep1_info_dict['Cell cycle state'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        if np.array([_v[_cind]==[] for _k,_v in cell_cycle_flag_dict.items()]).any():
            for _k,_v in cell_cycle_flag_dict.items():
                if _k == _state:
                    _v[_cind] = True
                else:
                    _v[_cind] = False
    # append to data
    for _k, _v in cell_cycle_flag_dict.items():
        data_rep1[f'{_k}_flags'] = np.array(_v)  

['Z(nm)', 'X(nm)', 'Y(nm)', 'Genomic coordinate', 'Chromosome copy number', 'Gene names', 'Transcription', 'TSS ZXY(nm)']


0it [00:00, ?it/s]

84 genes exist in this dataset.


0it [00:00, ?it/s]

/net/gs/vol1/home/gesine/local/anaconda3/envs/su2020/lib/python3.7/site-packages/ipykernel_launcher.py:101: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


<a id='1.2'></a>
## 1.2 load chr21 replicate 2 

(with cell cycle information)

In [107]:
# load from file and extract info
import csv
rep2_info_dict = {}
with open(rep2_filename, 'r') as _handle:
    _reader = csv.reader(_handle, delimiter='\t', quotechar='|')
    _headers = next(_reader)
    print(_headers)
    # create keys for each header
    for _h in _headers:
        rep2_info_dict[_h] = []
    # loop through content
    for _contents in _reader:
        for _h, _info in zip(_headers,_contents):
            rep2_info_dict[_h].append(_info)

# clean up information
data_rep2 = {'params':{}}

# clean up genomic coordiantes
region_names = np.array([_n for _n in sorted(np.unique(rep2_info_dict['Genomic coordinate']), 
                                             key=lambda s:int(s.split(':')[1].split('-')[0]))])
region_starts = np.array([int(_n.split(':')[1].split('-')[0]) for _n in region_names])
region_ends = np.array([int(_n.split(':')[1].split('-')[1]) for _n in region_names])[np.argsort(region_starts)]
region_starts = np.sort(region_starts)

mid_positions = ((region_starts + region_ends)/2).astype(int)
mid_positions_Mb = np.round(mid_positions / 1e6, 5) 
start_position_Mb = np.round(region_starts / 1e6, 5) 
end_position_Mb = np.round(region_ends / 1e6, 5) 

# clean up chrom copy number
chr_nums = np.array([int(_info) for _info in rep2_info_dict['Chromosome copy number']])
chr_ids, region_cts = np.unique(chr_nums, return_counts=True)
dna_zxys_list = [[[] for _start in region_starts] for _id in chr_ids]

# clean up zxy
for _z,_x,_y,_reg_info, _cid in tqdm(zip(rep2_info_dict['Z(nm)'],rep2_info_dict['X(nm)'],\
                                         rep2_info_dict['Y(nm)'],rep2_info_dict['Genomic coordinate'],\
                                         rep2_info_dict['Chromosome copy number'])):
    # get chromosome inds
    _cid = int(_cid)
    _cind = np.where(chr_ids == _cid)[0][0]
    
    # get region indices
    _start = int(_reg_info.split(':')[1].split('-')[0])
    _rind = np.where(region_starts==_start)[0][0]
    
    dna_zxys_list[_cind][_rind] = np.array([float(_z),float(_x), float(_y)])

# merge together
dna_zxys_list = np.array(dna_zxys_list)
data_rep2['chrom_ids'] = chr_ids
data_rep2['region_names'] = region_names
data_rep2['mid_position_Mb'] = mid_positions_Mb
data_rep2['start_position_Mb'] = start_position_Mb
data_rep2['end_position_Mb'] = end_position_Mb
data_rep2['dna_zxys'] = dna_zxys_list

# clean up tss and transcription
if 'Gene names' in rep2_info_dict:
    import re
    # first extract number of genes
    gene_names = []
    for _gene_info, _trans_info, _tss_coord in zip(rep2_info_dict['Gene names'],
                                                   rep2_info_dict['Transcription'],
                                                   rep2_info_dict['TSS ZXY(nm)']):
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            for _gene in _genes:
                if _gene not in gene_names:
                    gene_names.append(_gene)
    print(f"{len(gene_names)} genes exist in this dataset.")
    
    # initialize gene and transcription
    tss_zxys_list = [[[] for _gene in gene_names] for _id in chr_ids]
    transcription_profiles = [[[] for _gene in gene_names] for _id in chr_ids]
    # loop through to get info
    for _cid, _gene_info, _trans_info, _tss_locations in tqdm(zip(rep2_info_dict['Chromosome copy number'],
                                                                  rep2_info_dict['Gene names'],
                                                                  rep2_info_dict['Transcription'],
                                                                  rep2_info_dict['TSS ZXY(nm)'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        # process if there are genes in this region:
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            _transcribes = _trans_info.split(';')[:-1]
            _tss_zxys = _tss_locations.split(';')[:-1]
            for _gene, _transcribe, _tss_zxy in zip(_genes, _transcribes, _tss_zxys):
                # get gene index
                _gind = gene_names.index(_gene)
                # get transcription profile
                if _transcribe == 'on':
                    transcription_profiles[_cind][_gind] = True
                else:
                    transcription_profiles[_cind][_gind] = False
                # get coordinates
                _tss_zxy = np.array([np.float(_c) for _c in re.split(r'\s+', _tss_zxy.split('[')[1].split(']')[0]) if _c != ''])
                tss_zxys_list[_cind][_gind] = _tss_zxy
                
    tss_zxys_list = np.array(tss_zxys_list)
    transcription_profiles = np.array(transcription_profiles)
    data_rep2['gene_names'] = gene_names
    data_rep2['tss_zxys'] = tss_zxys_list
    data_rep2['trans_pfs'] = transcription_profiles

# clean up cell_cycle states
if 'Cell cycle state' in rep2_info_dict:
    cell_cycle_types = np.unique(rep2_info_dict['Cell cycle state'])
    cell_cycle_flag_dict = {_k:[[] for _id in chr_ids] for _k in cell_cycle_types if _k != 'ND'}
    for _cid, _state in tqdm(zip(rep2_info_dict['Chromosome copy number'],rep2_info_dict['Cell cycle state'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        if np.array([_v[_cind]==[] for _k,_v in cell_cycle_flag_dict.items()]).any():
            for _k,_v in cell_cycle_flag_dict.items():
                if _k == _state:
                    _v[_cind] = True
                else:
                    _v[_cind] = False
    # append to data
    for _k, _v in cell_cycle_flag_dict.items():
        data_rep2[f'{_k}_flags'] = np.array(_v)  

['Z(nm)', 'X(nm)', 'Y(nm)', 'Genomic coordinate', 'Chromosome copy number', 'Gene names', 'Transcription', 'TSS ZXY(nm)', 'Cell cycle state']


0it [00:00, ?it/s]

84 genes exist in this dataset.


0it [00:00, ?it/s]

/net/gs/vol1/home/gesine/local/anaconda3/envs/su2020/lib/python3.7/site-packages/ipykernel_launcher.py:101: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


0it [00:00, ?it/s]

<a id='1.3'></a>
## 1.3 combine two datasets

in most of panels we combined chromosomes from two replicates for better statistics

In [108]:
data_combined = {}
for _k in data_rep1:
    if _k in data_rep2:
        # concatenate if this is some chromosomal data
        if len(data_rep1[_k]) == len(data_rep1["chrom_ids"]):
            data_combined[_k] = np.concatenate([data_rep1[_k],data_rep2[_k]])
        # directly merge if this is some shared information
        elif (np.array(data_rep1[_k]) == np.array(data_rep2[_k])).all():
            data_combined[_k] = data_rep1[_k]
            
print('Number of chromosomes:', len(data_combined['chrom_ids']))

Number of chromosomes: 12149


<a id='1.3'></a>
## 1.gc Save current progress

In [109]:
all_data = {chrom: data_combined, rep1: data_rep1, rep2: data_rep2}

In [227]:
# [GC+] Assess data structure
in_common = False
for name, data in all_data.items():
    print(f'\n======== {name} ========')
    for key, val in data.items():
        if key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key]):
            in_common = True
            continue
        if isinstance(val, np.ndarray):
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {val.shape}')
        else:
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {len(val)}')

if in_common:
    print(f'\n======== BOTH REPS ========')
    for key, val in data_rep1.items():
        if not (key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key])):
            continue
        if isinstance(val, np.ndarray):
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {val.shape}')
        else:
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {len(val)}')
            

print("\n\nIs 'chrom_ids' just an index, eg. np.arange(1, len(chrom_ids) + 1)'?")
for name, data in all_data.items():
    if 'chrom_ids' in data:
        tmp = np.array_equal(data_rep1['chrom_ids'], np.arange(1, len(data_rep1['chrom_ids']) + 1))
        print(f"{name.ljust(25, ' ')} " + {True: '✓', False: '✗'}[tmp])

print("\n\nConfirm there aren't any weird rounding errors in 'start_position_Mb':")
for name, data in all_data.items():
    if 'start_position_Mb' in data:
        tmp = (data['start_position_Mb'] - np.round(data['start_position_Mb'], 4)) != 0
        print(f"{name.ljust(25, ' ')} " + {True: '✓', False: '✗'}[~np.any(tmp)])


======== chr21 ========
chrom_ids            ndarray    (12149,)
dna_zxys             ndarray    (12149, 651, 3)
tss_zxys             ndarray    (12149, 84, 3)
trans_pfs            ndarray    (12149, 84)

======== chr21.rep1 ========
chrom_ids            ndarray    (7591,)
dna_zxys             ndarray    (7591, 651, 3)
tss_zxys             ndarray    (7591, 84, 3)
trans_pfs            ndarray    (7591, 84)

======== chr21.rep2_cell_cycle ========
chrom_ids            ndarray    (4558,)
dna_zxys             ndarray    (4558, 651, 3)
tss_zxys             ndarray    (4558, 84, 3)
trans_pfs            ndarray    (4558, 84)
G1_flags             ndarray    (4558,)
G2/S_flags           ndarray    (4558,)

======== BOTH REPS ========
params               dict       0
region_names         ndarray    (651,)
mid_position_Mb      ndarray    (651,)
start_position_Mb    ndarray    (651,)
end_position_Mb      ndarray    (651,)
gene_names           list       84


Is 'chrom_ids' just an index, eg. np

In [206]:
print((region_starts)[:10])
print((region_starts - 1)[:10])
print(((region_starts - 1)[:10].astype(int) / 10 ** 6))
print(np.round((region_starts - 1)[:10].astype(int) / 10 ** 6, 4) % 0.05)


[10400001 10500001 10600001 13250001 14000001 14050001 14100001 14150001
 14200001 14250001]
[10400000 10500000 10600000 13250000 14000000 14050000 14100000 14150000
 14200000 14250000]
[10.4  10.5  10.6  13.25 14.   14.05 14.1  14.15 14.2  14.25]
[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05]


In [232]:
# [GC+] SAVE in-progress files - numpy

for name, data in all_data.items():
    for key, val in data.items():
        if len(val) == 0 or key in ('start_position_Mb', 'end_position_Mb') or np.array_equal(np.asarray(val).ravel(), np.arange(np.asarray(val).size) + 1):
            continue
        if key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key]):
            tmp = chrom
        else:
            tmp = name
        if isinstance(val, dict):
            pass
        else:
            key = key.replace('/', '-')
            print(os.path.join(dir_3d_coords, f"{tmp}.{key}.npy"))
            np.save(os.path.join(dir_3d_coords, f"{tmp}.{key}.npy"), val)

/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.chrom_ids.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.region_names.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.mid_position_Mb.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.dna_zxys.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.gene_names.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.tss_zxys.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr21/chr21.trans_pfs.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Ana

In [210]:
# [GC+] SAVE in-progress files - pickled
with open(os.path.join(dir_3d_coords, rep1) + '.pickle', 'wb') as handle:
    pickle.dump(data_rep1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(dir_3d_coords, rep2) + '.pickle', 'wb') as handle:
    pickle.dump(data_rep2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(dir_3d_coords, chrom) + '.pickle', 'wb') as handle:
    pickle.dump(data_combined, handle, protocol=pickle.HIGHEST_PROTOCOL)

<a id='1.3'></a>
## 1.gc Load in-progress files

In [ ]:
# # [GC+] LOAD in-progress files

# with open(rep1_coords_file, 'rb') as handle:
#     data_rep1 = pickle.load(handle)
# with open(rep2_coords_file, 'rb') as handle:
#     data_rep2 = pickle.load(handle)

# with open(combo_coords_file, 'rb') as handle:
#     data_combined = pickle.load(handle)

<a id='2.1'></a>
## 2.1 generate imaging-based median distance and proximity frequency maps

In [5]:
# from scipy.spatial.distance import pdist, squareform
# zxys_combined_list = np.array(data_combined['dna_zxys'])
# distmap_combined_list = np.array([squareform(pdist(_zxy)) for _zxy in tqdm(zxys_combined_list)])

# # generate median distance map
# median_distance_map_combined = np.nanmedian(distmap_combined_list, axis = 0)
# # generate contact map
# contact_th = 500
# contact_map_combined = np.sum(distmap_combined_list<contact_th, axis=0) / np.sum(np.isnan(distmap_combined_list)==False, axis=0)

In [ ]:
# [GC+] edited so I can SAVE in-progress files

from scipy.spatial.distance import pdist, squareform

zxys_combined_list = np.array(data_combined['dna_zxys'])

distmap_combined_list = np.concatenate([pdist(_zxy).reshape(1, -1) for _zxy in tqdm(zxys_combined_list)])
np.save(distmap_list_file, distmap_combined_list) # [GC+] SAVE in-progress files

  0%|          | 0/12149 [00:00<?, ?it/s]

  0%|          | 0/12149 [00:00<?, ?it/s]

In [28]:
chrom = 'chr21'
distvec_per_cell = None
print('Loading sc distances...'); distvec_per_cell = np.load(files_progress[chrom]['distances.per_cell'])
print(chrom)
process_sc_distances(files_progress[chrom], distvec_per_cell=distvec_per_cell, dis_mean=True, dis_median=True, contact_th=500)

Loading sc distances...
chr21


<a id='1.3'></a>
## 2.gc Load in-progress files

In [ ]:
# # [GC+] LOAD in-progress files
# distmap_combined_list = np.load(distmap_list_file)
# median_distance_map_combined = np.load(median_distance_map_file)
# contact_map_combined = np.load(median_distance_map_file)

<a id='1'></a>
# Chromosome 2

<a id='1.gc'></a>
## 1.gc Filenames

In [9]:
chrom = 'chr2'
rep1_filename = files_3dcoords[chrom]['rep1']
rep2_filename = files_3dcoords[chrom]['rep2']
rep1 = files_progress[chrom]['rep1']
rep2 = files_progress[chrom]['rep2']
print(chrom); print(rep1); print(rep2)

dir_3d_coords =  files_progress[chrom]['dir_coords3d']
distmap_list_file = files_progress[chrom]['distances.per_cell']
median_distance_map_file = files_progress[chrom]['distances.median']
mean_distance_map_file = files_progress[chrom]['distances.mean']
contacts_mean_c500_file = files_progress[chrom]['contacts.mean.cutoff500']
contacts_percell_c500_file = files_progress[chrom]['contacts.per_cell.cutoff500']

chr2
chr2.rep1
chr2.rep2_p_arm


<a id='1.1'></a>
## 1.1 load chr2 replicate 1 

(entire chr2)

In [87]:
# load from file and extract info
import csv
rep1_info_dict = {}
with open(rep1_filename, 'r') as _handle:
    _reader = csv.reader(_handle, delimiter='\t', quotechar='|')
    _headers = next(_reader)
    print(_headers)
    # create keys for each header
    for _h in _headers:
        rep1_info_dict[_h] = []
    # loop through content
    for _contents in _reader:
        for _h, _info in zip(_headers,_contents):
            rep1_info_dict[_h].append(_info)

# clean up information
data_rep1 = {'params':{}}

# clean up genomic coordiantes
region_names = np.array([_n for _n in sorted(np.unique(rep1_info_dict['Genomic coordinate']), 
                                             key=lambda s:int(s.split(':')[1].split('-')[0]))])
region_starts = np.array([int(_n.split(':')[1].split('-')[0]) for _n in region_names])
region_ends = np.array([int(_n.split(':')[1].split('-')[1]) for _n in region_names])[np.argsort(region_starts)]
region_starts = np.sort(region_starts)

mid_positions = ((region_starts + region_ends)/2).astype(int)
mid_positions_Mb = np.round(mid_positions / 1e6, 5) 
start_position_Mb = np.round(region_starts / 1e6, 5) 
end_position_Mb = np.round(region_ends / 1e6, 5) 

# clean up chrom copy number
chr_nums = np.array([int(_info) for _info in rep1_info_dict['Chromosome copy number']])
chr_ids, region_cts = np.unique(chr_nums, return_counts=True)
dna_zxys_list = [[[] for _start in region_starts] for _id in chr_ids]

# clean up zxy
for _z,_x,_y,_reg_info, _cid in tqdm(zip(rep1_info_dict['Z(nm)'],rep1_info_dict['X(nm)'],\
                                         rep1_info_dict['Y(nm)'],rep1_info_dict['Genomic coordinate'],\
                                         rep1_info_dict['Chromosome copy number'])):
    # get chromosome inds
    _cid = int(_cid)
    _cind = np.where(chr_ids == _cid)[0][0]
    
    # get region indices
    _start = int(_reg_info.split(':')[1].split('-')[0])
    _rind = np.where(region_starts==_start)[0][0]
    
    dna_zxys_list[_cind][_rind] = np.array([float(_z),float(_x), float(_y)])

# merge together
dna_zxys_list = np.array(dna_zxys_list)
data_rep1['chrom_ids'] = chr_ids
data_rep1['region_names'] = region_names
data_rep1['mid_position_Mb'] = mid_positions_Mb
data_rep1['start_position_Mb'] = start_position_Mb
data_rep1['end_position_Mb'] = end_position_Mb
data_rep1['dna_zxys'] = dna_zxys_list

# clean up tss and transcription
if 'Gene names' in rep1_info_dict:
    import re
    # first extract number of genes
    gene_names = []
    for _gene_info, _trans_info, _tss_coord in zip(rep1_info_dict['Gene names'],
                                                   rep1_info_dict['Transcription'],
                                                   rep1_info_dict['TSS ZXY(nm)']):
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            for _gene in _genes:
                if _gene not in gene_names:
                    gene_names.append(_gene)
    print(f"{len(gene_names)} genes exist in this dataset.")
    
    # initialize gene and transcription
    tss_zxys_list = [[[] for _gene in gene_names] for _id in chr_ids]
    transcription_profiles = [[[] for _gene in gene_names] for _id in chr_ids]
    # loop through to get info
    for _cid, _gene_info, _trans_info, _tss_locations in tqdm(zip(rep1_info_dict['Chromosome copy number'],
                                                                  rep1_info_dict['Gene names'],
                                                                  rep1_info_dict['Transcription'],
                                                                  rep1_info_dict['TSS ZXY(nm)'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        # process if there are genes in this region:
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            _transcribes = _trans_info.split(';')[:-1]
            _tss_zxys = _tss_locations.split(';')[:-1]
            for _gene, _transcribe, _tss_zxy in zip(_genes, _transcribes, _tss_zxys):
                # get gene index
                _gind = gene_names.index(_gene)
                # get transcription profile
                if _transcribe == 'on':
                    transcription_profiles[_cind][_gind] = True
                else:
                    transcription_profiles[_cind][_gind] = False
                # get coordinates
                _tss_zxy = np.array([np.float(_c) for _c in re.split(r'\s+', _tss_zxy.split('[')[1].split(']')[0]) if _c != ''])
                tss_zxys_list[_cind][_gind] = _tss_zxy
                
    tss_zxys_list = np.array(tss_zxys_list)
    transcription_profiles = np.array(transcription_profiles)
    data_rep1['gene_names'] = gene_names
    data_rep1['tss_zxys'] = tss_zxys_list
    data_rep1['trans_pfs'] = transcription_profiles

# clean up cell_cycle states
if 'Cell cycle state' in rep1_info_dict:
    cell_cycle_types = np.unique(rep1_info_dict['Cell cycle state'])
    cell_cycle_flag_dict = {_k:[[] for _id in chr_ids] for _k in cell_cycle_types if _k != 'ND'}
    for _cid, _state in tqdm(zip(rep1_info_dict['Chromosome copy number'],rep1_info_dict['Cell cycle state'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        if np.array([_v[_cind]==[] for _k,_v in cell_cycle_flag_dict.items()]).any():
            for _k,_v in cell_cycle_flag_dict.items():
                if _k == _state:
                    _v[_cind] = True
                else:
                    _v[_cind] = False
    # append to data
    for _k, _v in cell_cycle_flag_dict.items():
        data_rep1[f'{_k}_flags'] = np.array(_v)  

['Z(nm)', 'X(nm)', 'Y(nm)', 'Genomic coordinate', 'Chromosome copy number']


0it [00:00, ?it/s]

<a id='1.2'></a>
## 1.2 load chr2 replicate 2 

(p-arm only, first 357 regions)

In [82]:
# load from file and extract info
import csv
rep2_info_dict = {}
with open(rep2_filename, 'r') as _handle:
    _reader = csv.reader(_handle, delimiter='\t', quotechar='|')
    _headers = next(_reader)
    print(_headers)
    # create keys for each header
    for _h in _headers:
        rep2_info_dict[_h] = []
    # loop through content
    for _contents in _reader:
        for _h, _info in zip(_headers,_contents):
            rep2_info_dict[_h].append(_info)

# clean up information
data_rep2 = {'params':{}}

# clean up genomic coordiantes
region_names = np.array([_n for _n in sorted(np.unique(rep2_info_dict['Genomic coordinate']), 
                                             key=lambda s:int(s.split(':')[1].split('-')[0]))])
region_starts = np.array([int(_n.split(':')[1].split('-')[0]) for _n in region_names])
region_ends = np.array([int(_n.split(':')[1].split('-')[1]) for _n in region_names])[np.argsort(region_starts)]
region_starts = np.sort(region_starts)

mid_positions = ((region_starts + region_ends)/2).astype(int)
mid_positions_Mb = np.round(mid_positions / 1e6, 5) 
start_position_Mb = np.round(region_starts / 1e6, 5) 
end_position_Mb = np.round(region_ends / 1e6, 5) 

# clean up chrom copy number
chr_nums = np.array([int(_info) for _info in rep2_info_dict['Chromosome copy number']])
chr_ids, region_cts = np.unique(chr_nums, return_counts=True)
dna_zxys_list = [[[] for _start in region_starts] for _id in chr_ids]

# clean up zxy
for _z,_x,_y,_reg_info, _cid in tqdm(zip(rep2_info_dict['Z(nm)'],rep2_info_dict['X(nm)'],\
                                         rep2_info_dict['Y(nm)'],rep2_info_dict['Genomic coordinate'],\
                                         rep2_info_dict['Chromosome copy number'])):
    # get chromosome inds
    _cid = int(_cid)
    _cind = np.where(chr_ids == _cid)[0][0]
    
    # get region indices
    _start = int(_reg_info.split(':')[1].split('-')[0])
    _rind = np.where(region_starts==_start)[0][0]
    
    dna_zxys_list[_cind][_rind] = np.array([float(_z),float(_x), float(_y)])

# merge together
dna_zxys_list = np.array(dna_zxys_list)
data_rep2['chrom_ids'] = chr_ids
data_rep2['region_names'] = region_names
data_rep2['mid_position_Mb'] = mid_positions_Mb
data_rep2['start_position_Mb'] = start_position_Mb
data_rep2['end_position_Mb'] = end_position_Mb
data_rep2['dna_zxys'] = dna_zxys_list

# clean up tss and transcription
if 'Gene names' in rep2_info_dict:
    import re
    # first extract number of genes
    gene_names = []
    for _gene_info, _trans_info, _tss_coord in zip(rep2_info_dict['Gene names'],
                                                   rep2_info_dict['Transcription'],
                                                   rep2_info_dict['TSS ZXY(nm)']):
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            for _gene in _genes:
                if _gene not in gene_names:
                    gene_names.append(_gene)
    print(f"{len(gene_names)} genes exist in this dataset.")
    
    # initialize gene and transcription
    tss_zxys_list = [[[] for _gene in gene_names] for _id in chr_ids]
    transcription_profiles = [[[] for _gene in gene_names] for _id in chr_ids]
    # loop through to get info
    for _cid, _gene_info, _trans_info, _tss_locations in tqdm(zip(rep2_info_dict['Chromosome copy number'],
                                                                  rep2_info_dict['Gene names'],
                                                                  rep2_info_dict['Transcription'],
                                                                  rep2_info_dict['TSS ZXY(nm)'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        # process if there are genes in this region:
        if _gene_info != '':
            # split by semicolon
            _genes = _gene_info.split(';')[:-1]
            _transcribes = _trans_info.split(';')[:-1]
            _tss_zxys = _tss_locations.split(';')[:-1]
            for _gene, _transcribe, _tss_zxy in zip(_genes, _transcribes, _tss_zxys):
                # get gene index
                _gind = gene_names.index(_gene)
                # get transcription profile
                if _transcribe == 'on':
                    transcription_profiles[_cind][_gind] = True
                else:
                    transcription_profiles[_cind][_gind] = False
                # get coordinates
                _tss_zxy = np.array([np.float(_c) for _c in re.split(r'\s+', _tss_zxy.split('[')[1].split(']')[0]) if _c != ''])
                tss_zxys_list[_cind][_gind] = _tss_zxy
                
    tss_zxys_list = np.array(tss_zxys_list)
    transcription_profiles = np.array(transcription_profiles)
    data_rep2['gene_names'] = gene_names
    data_rep2['tss_zxys'] = tss_zxys_list
    data_rep2['trans_pfs'] = transcription_profiles

# clean up cell_cycle states
if 'Cell cycle state' in rep2_info_dict:
    cell_cycle_types = np.unique(rep2_info_dict['Cell cycle state'])
    cell_cycle_flag_dict = {_k:[[] for _id in chr_ids] for _k in cell_cycle_types if _k != 'ND'}
    for _cid, _state in tqdm(zip(rep2_info_dict['Chromosome copy number'],rep2_info_dict['Cell cycle state'])):
        # get chromosome inds
        _cid = int(_cid)
        _cind = np.where(chr_ids == _cid)[0][0]
        if np.array([_v[_cind]==[] for _k,_v in cell_cycle_flag_dict.items()]).any():
            for _k,_v in cell_cycle_flag_dict.items():
                if _k == _state:
                    _v[_cind] = True
                else:
                    _v[_cind] = False
    # append to data
    for _k, _v in cell_cycle_flag_dict.items():
        data_rep2[f'{_k}_flags'] = np.array(_v)  

['Z(nm)', 'X(nm)', 'Y(nm)', 'Genomic coordinate', 'Chromosome copy number']


0it [00:00, ?it/s]

<a id='1.3'></a>
## 1.gc Save current progress

In [91]:
all_data = {rep1: data_rep1, rep2: data_rep2}

In [92]:
# [GC+] Assess data structure
in_common = False
for name, data in all_data.items():
    print(f'\n======== {name} ========')
    for key, val in data.items():
        if key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key]):
            in_common = True
            continue
        if isinstance(val, np.ndarray):
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {val.shape}')
        else:
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {len(val)}')

if in_common:
    print(f'\n======== BOTH REPS ========')
    for key, val in data_rep1.items():
        if not (key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key])):
            continue
        if isinstance(val, np.ndarray):
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {val.shape}')
        else:
            print(f'{key.ljust(20, " ")} {type(val).__name__.ljust(10, " ")} {len(val)}')
            

print("\n\nIs 'chrom_ids' just an index, eg. np.arange(1, len(chrom_ids) + 1)'?")
for name, data in all_data.items():
    if 'chrom_ids' in data:
        tmp = np.array_equal(data_rep1['chrom_ids'], np.arange(1, len(data_rep1['chrom_ids']) + 1))
        print(f"{name.ljust(25, ' ')} " + {True: '✓', False: '✗'}[tmp])

print("\n\nConfirm there aren't any weird rounding errors in 'start_position_Mb':")
for name, data in all_data.items():
    if 'start_position_Mb' in data:
        tmp = (data['start_position_Mb'] - np.round(data['start_position_Mb'], 4)) != 0
        print(f"{name.ljust(25, ' ')} " + {True: '✓', False: '✗'}[~np.any(tmp)])


======== chr2.rep1 ========
chrom_ids            ndarray    (3029,)
region_names         ndarray    (935,)
mid_position_Mb      ndarray    (935,)
start_position_Mb    ndarray    (935,)
end_position_Mb      ndarray    (935,)
dna_zxys             ndarray    (3029, 935, 3)

======== chr2.rep2_p_arm ========
chrom_ids            ndarray    (4848,)
region_names         ndarray    (357,)
mid_position_Mb      ndarray    (357,)
start_position_Mb    ndarray    (357,)
end_position_Mb      ndarray    (357,)
dna_zxys             ndarray    (4848, 357, 3)

======== BOTH REPS ========
params               dict       0


Is 'chrom_ids' just an index, eg. np.arange(1, len(chrom_ids) + 1)'?
chr2.rep1           : True
chr2.rep2_p_arm     : True


Are there weird rounding errors in 'start_position_Mb'?
False
False


In [99]:
# [GC+] SAVE in-progress files - numpy

for name, data in all_data.items():
    for key, val in data.items():
        if len(val) == 0 or key in ('start_position_Mb', 'end_position_Mb') or np.array_equal(np.asarray(val).ravel(), np.arange(np.asarray(val).size) + 1):
            continue
        if key in data_rep1 and key in data_rep2 and np.array_equal(data_rep1[key], data_rep2[key]):
            tmp = chrom
        else:
            tmp = name
        if isinstance(val, dict):
            pass
        else:
            key = key.replace('/', '-')
            print(os.path.join(dir_3d_coords, f"{tmp}.{key}.npy"))
            np.save(os.path.join(dir_3d_coords, f"{tmp}.{key}.npy"), val)

/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep1.region_names.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep1.mid_position_Mb.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep1.dna_zxys.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep2_p_arm.region_names.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep2_p_arm.mid_position_Mb.npy
/net/noble/vol5/user/gesine/repos/Chromatin_Analysis_2020_cell/sequential_tracing/data/in_progress/3d_coords/chr2/chr2.rep2_p_arm.dna_zxys.npy


In [96]:
# [GC+] SAVE in-progress files - pickled
with open(os.path.join(dir_3d_coords, rep1) + '.pickle', 'wb') as handle:
    pickle.dump(data_rep1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(dir_3d_coords, rep2) + '.pickle', 'wb') as handle:
    pickle.dump(data_rep2, handle, protocol=pickle.HIGHEST_PROTOCOL)

<a id='1.3'></a>
## 1.gc Load in-progress files

In [ ]:
# # [GC+] LOAD in-progress files

# with open(rep1_coords_file, 'rb') as handle:
#     data_rep1 = pickle.load(handle)
# with open(rep2_coords_file, 'rb') as handle:
#     data_rep2 = pickle.load(handle)

## 2.0 corresponding regions for p and q arms

In [ ]:
# # P and q arm crop
# p_crop = slice(0, 357)
# q_crop = slice(357, len(data_rep1['dna_zxys'][0]))
# print(p_crop, q_crop)

<a id='2.1'></a>
## 2.1 generate imaging-based median distance and proximity frequency maps

In [ ]:
# from scipy.spatial.distance import squareform, pdist
# zxys_rep1_list = np.array(data_rep1['dna_zxys'])
# distmap_rep1_list = np.array([squareform(pdist(_zxy)) for _zxy in zxys_rep1_list])
# # calculate contact freq map
# contact_th = 500
# contact_rep1_map = np.sum(distmap_rep1_list<contact_th, axis=0) / np.sum(np.isnan(distmap_rep1_list)==False, axis=0)

In [100]:
# [GC+] edited so I can SAVE in-progress files

from scipy.spatial.distance import pdist, squareform

zxys_rep1_list = np.array(data_rep1['dna_zxys'])

distmap_rep1_list = np.concatenate([pdist(_zxy).reshape(1, -1) for _zxy in tqdm(zxys_rep1_list)])
np.save(distmap_list_file, distmap_rep1_list) # [GC+] SAVE in-progress files

  0%|          | 0/3029 [00:00<?, ?it/s]

  0%|          | 0/3029 [00:00<?, ?it/s]

In [ ]:
chrom = 'chr2'
print(chrom)
distvec_per_cell = None
print('Loading sc distances...'); distvec_per_cell = np.load(files_progress[chrom]['distances.per_cell'])
process_sc_distances(files_progress[chrom], distvec_per_cell=distvec_per_cell, dis_mean=True, dis_median=True, contact_th=500)

chr2
Loading sc distances...


<a id='1.3'></a>
## 2.gc Load in-progress files

In [ ]:
# # [GC+] LOAD in-progress files
# distmap_combined_list = np.load(distmap_list_file)
# median_distance_map_combined = np.load(median_distance_map_file)
# contact_map_combined = np.load(median_distance_map_file)

# Create proper hiclib-style counts matrices

In [58]:
# Quick check, make sure all regions are 0.05 Mb
for chrom in files_progress.keys():
    region_names = np.load(os.path.join(files_progress[chrom]['dir_coords3d'], files_progress[chrom]['proceed_with'] + '.region_names.npy'))
    regions = np.concatenate([np.array(re.split(r'[:-]', x)[1:], dtype=int).reshape(1, -1) for x in region_names])
    region_sizes = np.unique(regions[:,1] - regions[:,0]) / 1e6
    if region_sizes.size != 1 or region_sizes[0] != 0.05:
        raise ValueError('Region sizes not all equal to 0.05 Mb')
        
files_progress[chrom].keys()

dict_keys(['dir_coords3d', 'rep1', 'rep2', 'distances.per_cell', 'distances.median', 'distances.mean', 'contacts.cutoff500', 'proceed_with'])

In [59]:
# Quick check, make sure there aren't any beads that are NaN on one axis but not all axes
for chrom in files_progress.keys():
    dna_zxys = np.load(os.path.join(files_progress[chrom]['dir_coords3d'], files_progress[chrom]['proceed_with'] + '.dna_zxys.npy'))
    tmp = np.isnan(dna_zxys).sum(2) 
    irregular_nans = np.sum((tmp != 0) & (tmp != 3))
    if irregular_nans != 0:
        raise ValueError('Some beads have NaNs one one axis but not all axes!')

In [133]:
from scipy import sparse
import pandas as pd
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', message='', category=UserWarning)
    warnings.filterwarnings('ignore', message='', category=FutureWarning)
    from iced.io import write_counts, write_lengths

def get_complete_counts_matrix(files_progress, chrom, good_locus_th=0.75, good_cell_th=0.9, res=0.05, verbose=False, load_counts=True, save_counts=False):
    # Load data... shape=(ncells, nbins, 3)
    dna_zxys = np.load(os.path.join(files_progress[chrom]['dir_coords3d'], files_progress[chrom]['proceed_with'] + '.dna_zxys.npy'))
    ncells_orig = dna_zxys.shape[0]
    nbins_orig = dna_zxys.shape[1]
    
    # Ignore cells that have < good_cell_th*100% NaNs
    cells_good = np.mean(~np.isnan(dna_zxys[:, :, 0]), axis=1) >= good_cell_th # Ratio of good-bins-per-cell >= good_cell_th 
    perc_cells_discarded = (1 - np.mean(cells_good)) * 100
    dna_zxys = dna_zxys[cells_good, :, :] # Filter out 'bad cells'
    
    # Ignore bins that are NaN < good_locus_th*100% 
    success_rates = np.mean(~np.isnan(dna_zxys[:, :, 0]), axis=0)
    good_regions = np.where(success_rates >= good_locus_th)[0]
    perc_regions_discarded = (1 - np.mean(success_rates >= good_locus_th)) * 100

    # Get mids that pass cutoff
    mids_orig = np.load(os.path.join(files_progress[chrom]['dir_coords3d'], files_progress[chrom]['proceed_with'] + '.mid_position_Mb.npy'))
    if not np.array_equal(mids_orig, np.sort(mids_orig)):
        raise ValueError('Midpoints not in ascending order')
    mids_good = mids_orig[good_regions]
    
    # Compute coverage
    mids_all = np.round(np.arange(mids_good.min(), mids_good.max() + res, res), 6)
    coverage = mids_good.size / mids_all.size * 100
    idx = np.where(np.isin(mids_all, mids_good))[0]
    mids_all_untrunc = np.round(np.arange(mids_orig.min(), mids_orig.max() + res, res), 6)
    coverage_untrunc = mids_good.size / mids_all_untrunc.size * 100
    idx_untrunc = np.where(np.isin(mids_all_untrunc, mids_good))[0]

    if verbose:
        print(f"Chrom={chrom.ljust(9)} CellTh={str(good_cell_th).ljust(7)} LocusTh={str(good_locus_th).ljust(7)} Cov={coverage:.1f}%   CovUntrunc={coverage_untrunc:.1f}%")
        
    res = {'chrom': chrom, 'good_locus_th': good_locus_th, 'good_cell_th': good_cell_th,
           'coverage': coverage, 'coverage_untrunc': coverage_untrunc,
           'perc_cells_discarded': perc_cells_discarded, 'perc_regions_discarded': perc_regions_discarded, 'ncells': ncells_orig, 'nbins': nbins_orig}
        
    if load_counts:
        # Get counts that pass cutoff
        counts_orig = np.triu(np.load(os.path.join(files_progress[chrom]['contacts.mean.cutoff500'])), 1)
        counts_good = counts_orig[good_regions][:,good_regions]
        
        res['nreads_total'] = counts_good.sum() # FIXME - not correct if good_cell_th>0
        res['nreads_perbin'] = counts_good[np.triu_indices(counts_good.shape[0], 1)].mean() # FIXME - not correct if good_cell_th>0

        # Create complete counts matrix
        counts = np.zeros((mids_all.size, mids_all.size), dtype=int)
        counts[idx][:,idx] = counts_good
        
        # counts_good_nan = (counts_good.sum(axis=0).ravel() == 0) & (counts_good.sum(axis=1).ravel() == 0)
        # print(counts_good_nan.sum())
        #print(f"SHAPE: dna_zxys={dna_zxys.shape[1]}, mids_orig={mids_orig.size}, mids_good={mids_good.size}, mids_all={mids_all.size}, counts_orig={counts_orig.shape}, mids_all_untrunc={mids_all_untrunc.size}")
        
        if save_counts:
            outdir = os.path.join(counts_folder, f"{chrom}.cutoff{good_locus_th}")
            write_counts(os.path.join(outdir, "counts.matrix"), counts)
            write_lengths(os.path.join(outdir, "counts.bed"), np.array([counts.shape[0]]))
    
    return res

In [134]:
_ = get_complete_counts_matrix(files_progress, chrom='chr21', good_cell_th=0, good_locus_th=0, verbose=False)

In [135]:
list_good_cell_th = (0, 0.9, 0.99, 0.999)
list_good_locus_th = (0, 0.75, 0.8, 0.9, 0.95, 0.975, 0.98)

list_good_cell_th = (0.99, 0.999)
list_good_locus_th = (0.99,)

# list_good_cell_th = (0,)
# list_good_locus_th = (0,)

df = []
for chrom in files_progress.keys():
    for good_cell_th in list_good_cell_th:
        #print(f"\n=== good_cell_th={good_cell_th}")
        for good_locus_th in list_good_locus_th:
            res = get_complete_counts_matrix(files_progress, chrom=chrom, good_cell_th=good_cell_th, good_locus_th=good_locus_th, verbose=False)
            df.append(res)

df = pd.DataFrame(df)

In [136]:
df

,chrom,good_locus_th,good_cell_th,coverage,coverage_untrunc,perc_cells_discarded,perc_regions_discarded,ncells,nbins,nreads_total,nreads_perbin
0,chr21,0.99,0.990,88.639761,81.680441,82.895712,8.90937,12149,651,23600.730551,0.134456
1,chr21,0.99,0.999,89.669421,89.669421,98.617170,0.00000,12149,651,28191.154977,0.133244
2,chr2,0.99,0.990,19.314191,19.314191,3.202377,0.00000,3029,935,16843.182788,0.038574
3,chr2,0.99,0.999,19.314191,19.314191,13.040607,0.00000,3029,935,16843.182788,0.038574


In [172]:
chrom = 'chr2'
res = 0.05

mids_orig = np.load(os.path.join(files_progress[chrom]['dir_coords3d'], files_progress[chrom]['proceed_with'] + '.mid_position_Mb.npy'))

In [ ]:
foo = (np.round(mids_orig * 1e3)).astype(int)
to_next_bin = ((foo[1:] - foo[:-1]) / int(res * 1e3)).astype(int)
# gaps_mask = np.where(to_next_bin != 250)[0]
# print(np.stack([gaps_mask, to_next_bin[gaps_mask]], axis=1))
# # print(to_next_bin[gaps_mask])

print((foo[1:] - foo[:-1]).min() / int(res * 1e3))
(foo[1:] - foo[:-1]).min(), int(res * 1e3)

In [173]:
to_next_bin = ((mids_orig[1:] - mids_orig[:-1]) / res)
# gaps_mask = np.where(to_next_bin != 250)[0]
# print(np.stack([gaps_mask, to_next_bin[gaps_mask]], axis=1))
# # print(to_next_bin[gaps_mask])

# print((foo[1:] - foo[:-1]).min() / int(res *e3))
# (foo[1:] - foo[:-1]).min(), int(res * 1e3)

to_next_bin

array([ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5., 10.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5

array([[0.025],
       [0.275],
       [0.525],
       [0.775],
       [1.025],
       [1.275],
       [1.525],
       [1.775],
       [2.025],
       [2.275]])

In [ ]:
mids_good = mids_orig
mids_all = np.round(np.arange(mids_good.min(), mids_good.max() + res, res), 6)
gaps_mask, mids_all.shape, mids_good.shape


# # P and q arm crop
# p_crop = slice(0, 357)
# q_crop = slice(357, len(data_rep1['dna_zxys'][0]))
# print(p_crop, q_crop)

In [130]:
slice(0, 10)

slice(0, 10, None)